In [8]:
from qiskit import (
    QuantumRegister,
    ClassicalRegister,
    QuantumCircuit,
    execute,
    Aer
)
from qiskit.visualization import plot_histogram
from IPython.display import display

In [2]:
def dualcz(qc,i,j):
    qc.h(i)
    qc.cx(i,j)
    qc.h(i)
    
    
def build_extract_syndromes(qc):
    # reset register 5 to be |0>

    for i in range(4):
        dualcz(qc,i,5)
        qc.cz((i+1)%5, 5)
        qc.cz((i+2)%5, 5)
        dualcz(qc,(i+3)%5,5)
        qc.measure([5],[i])
        
        
def build_syndrome(qc, n):
    qc.reset(qr[6])
    qc.reset(qr[6])
    qc.h(6)
    dualcz(qc,(n)%5,5)
    qc.cx(6,5)
    qc.cx((1+n)%5,5)
    qc.cx((2+n)%5,5)
    qc.cx(6,5)
    dualcz(qc,(3+n)%5,5)
    qc.h(6)
    qc.measure([5],[0])
    qc.measure([6],[1])
    
    
def build_circuit(qc):
    build_syndrome(qc, 0)
    qc.append(extract_syndrome).c_if(cr, 1)
    qc.append(extract_syndrome).c_if(cr, 2)
    qc.append(extract_syndrome).c_if(cr, 3)
        

In [10]:
# dual Control-Z: H CX H
dualcz_reg = QuantumRegister(2)
dczc = QuantumCircuit(dualcz_reg, name="dczc")
dczc.h(0)
dczc.cx(0,1)
dczc.h(0)

dualcz = dczc.to_instruction()
    
# extract syndromes for two-bit [[5,1,3]] QEC scheme
esr = QuantumRegister(5, 'q')
esra = QuantumRegister(1, 'a')
esrc = ClassicalRegister(4, 'c')
esc = QuantumCircuit(esr, esra, esrc, name="esc")

for i in range(4):
    esc.append(dualcz, [esr[i], esra[0]])
    esc.cz(esr[(i+1)%5], esra[0])
    esc.cz(esr[(i+2)%5], esra[0])
    esc.append(dualcz, [esr[(i+3)%5], esra[0]])
    esc.measure([esra[0]],[esrc[i]])

extract_syndromes = esc.to_instruction()
        
# The full circuit


def build_syndrome(qc, qr, anc, cr, n):
    qc.h(anc[1])
    qc.append(dualcz,[qr[(n)%5],anc[0]])
    qc.cx(anc[1],anc[0])
    qc.cx(qr[(1+n)%5],anc[0])
    qc.cx(qr[(2+n)%5],anc[0])
    qc.cx(anc[1],anc[0])
    qc.append(dualcz,[qr[(3+n)%5],anc[0]])
    qc.h(anc[1])
    qc.measure([anc[0]],[cr[0]])
    qc.measure([anc[1]],[cr[1]])
    
    
    
    
simulator = Aer.get_backend('qasm_simulator')    

qr = QuantumRegister(5, 'q')
anc0 = QuantumRegister(2, 'a0')
cr0 = ClassicalRegister(2, 'c0')

def run_syndrome(qr, anc, cr, n):

    syndrome = QuantumCircuit(qr, anc, cr)
    
    build_syndrome(syndrome, qr, anc, cr, n)
    # display(syndrome.draw())
    job = execute(syndrome, simulator, shots=1)
    result = job.result()
    counts = result.get_counts(syndrome)

    flags = ([*counts][0])
    
    print("Error syndrome and flag for syndrome", n, ": ", flags)

    if flags == '00':
        if n == 3:
            return 0
        else:
            return run_syndrome(qr, QuantumRegister(2, 'a' + str(n+1)), ClassicalRegister(2, 'c' + str(n+1)), n+1)
    
    else:
        ancE = QuantumRegister(1, 'aE')
        crE = ClassicalRegister(4, 'cE')
        
        extract = QuantumCircuit(qr, ancE, crE)
        extract.append(extract_syndromes, [qr[0], qr[1], qr[2], qr[3], qr[4], ancE[0]], [crE[0], crE[1], crE[2], crE[3]])
        
        job = execute(extract, simulator, shots=1)
        result = job.result()
        counts = result.get_counts(extract)
        
        syndrome_code = ([*counts][0])
        
        print("Syndrome code: ", syndrome_code)
        
        fixer = QuantumCircuit(qr)
        
        if flags == '10':
            
            if syndrome_code == '0001':
                fixer.x(qr[0])
            if syndrome_code == '1011':
                fixer.x(qr[0])
                fixer.z(qr[0])
            if syndrome_code == '1010':
                fixer.z(qr[0])
            if syndrome_code == '1000':
                fixer.x(qr[1])
            if syndrome_code == '1101':
                fixer.x(qr[1])
                fixer.z(qr[1])
            if syndrome_code == '0101':
                fixer.z(qr[1])
            if syndrome_code == '1100':
                fixer.x(qr[2])
            if syndrome_code == '1110':
                fixer.x(qr[2])
                fixer.z(qr[2])
            if syndrome_code == '0010':
                fixer.z(qr[2])
            if syndrome_code == '0110':
                fixer.x(qr[3])
            if syndrome_code == '1111':
                fixer.x(qr[3])
                fixer.z(qr[3])
            if syndrome_code == '1001':
                fixer.z(qr[3])
            if syndrome_code == '0011':
                fixer.x(qr[4])
            if syndrome_code == '0111':
                fixer.x(qr[4])
                fixer.z(qr[4])
            if syndrome_code == '0100':
                fixer.z(qr[4])
                
        else:
            
            if n == 0:
                
                if syndrome_code == '0001':
                    fixer.x(qr[0])
                if syndrome_code == '0100':
                    fixer.x(qr[3])
                    fixer.z(qr[2])
                if syndrome_code == '0110':
                    fixer.x(qr[3])
                if syndrome_code == '1000':
                    fixer.x(qr[2])
                    fixer.x(qr[3])
                    fixer.z(qr[2])
                if syndrome_code == '1001':
                    fixer.x(qr[0])
                    fixer.x(qr[1])
                if syndrome_code == '1010':
                    fixer.x(qr[2])
                    fixer.x(qr[3])
                if syndrome_code == '1100':
                    fixer.x(qr[3])
                    fixer.x(qr[4])
                    fixer.z(qr[3])
                
            if n == 1:
                
                if syndrome_code == '0011':
                    fixer.x(qr[4])
                if syndrome_code == '0100':
                    fixer.x(qr[1])
                    fixer.x(qr[2])
                if syndrome_code == '0101':
                    fixer.x(qr[3])
                    fixer.x(qr[4])
                if syndrome_code == '0110':
                    fixer.x(qr[0])
                    fixer.x(qr[4])
                    fixer.z(qr[4])
                if syndrome_code == '1000':
                    fixer.x(qr[1])
                if syndrome_code == '1010':
                    fixer.x(qr[4])
                    fixer.z(qr[3])
                if syndrome_code == '1100':
                    fixer.x(qr[3])
                    fixer.x(qr[4])
                    fixer.z(qr[3])
                
            if n == 2:
                
                if syndrome_code == '0001':
                    fixer.x(qr[0])
                if syndrome_code == '0010':
                    fixer.x(qr[0])
                    fixer.x(qr[4])
                if syndrome_code == '0011':
                    fixer.x(qr[0])
                    fixer.x(qr[1])
                    fixer.z(qr[0])
                if syndrome_code == '0101':
                    fixer.x(qr[0])
                    fixer.z(qr[4])
                if syndrome_code == '0110':
                    fixer.x(qr[0])
                    fixer.x(qr[4])
                    fixer.x(qr[4])
                if syndrome_code == '1010':
                    fixer.x(qr[2])
                    fixer.x(qr[3])
                if syndrome_code == '1100':
                    fixer.x(qr[2])
                
            if n == 3:
                
                if syndrome_code == '0001':
                    fixer.x(qr[3])
                    fixer.x(qr[4])
                    fixer.z(qr[4])
                if syndrome_code == '0010':
                    fixer.x(qr[1])
                    fixer.z(qr[0])
                if syndrome_code == '0011':
                    fixer.x(qr[0])
                    fixer.x(qr[1])
                    fixer.z(qr[0])
                if syndrome_code == '0101':
                    fixer.x(qr[3])
                    fixer.x(qr[4])
                if syndrome_code == '0110':
                    fixer.x(qr[3])
                if syndrome_code == '1000':
                    fixer.x(qr[1])
                if syndrome_code == '1001':
                    fixer.x(qr[0])
                    fixer.x(qr[1])
                
                
        return 1

error_detected = run_syndrome(qr, anc0, cr0, 0)

if error_detected:
    print("Error detected and fixed.")
else:
    print("No error detected.")

┌───────┐                                   
 q_0: ┤0      ├───────────────────────────────────
      │       │                                   
 q_1: ┤       ├───────■───────────────────────────
      │       │       │                           
 q_2: ┤       ├───────┼────■──────────────────────
      │  dczc │       │    │       ┌───────┐      
 q_3: ┤       ├───────┼────┼───────┤0      ├──────
      │       │       │    │       │       │      
 q_4: ┤       ├───────┼────┼───────┤  dczc ├──────
      │       │┌───┐┌─┴─┐┌─┴─┐┌───┐│       │┌─┐   
a0_0: ┤1      ├┤ X ├┤ X ├┤ X ├┤ X ├┤1      ├┤M├───
      └─┬───┬─┘└─┬─┘└───┘└───┘└─┬─┘└─┬───┬─┘└╥┘┌─┐
a0_1: ──┤ H ├────■──────────────■────┤ H ├───╫─┤M├
        └───┘                        └───┘   ║ └╥┘
c0: 2/═══════════════════════════════════════╩══╩═
                                             0  1

Error syndrome and flag for syndrome 0 :  00


q_0: ────────────────────────────────────────────
      ┌───────┐                                   
 q_1: ┤0      ├───────────────────────────────────
      │       │                                   
 q_2: ┤       ├───────■───────────────────────────
      │       │       │                           
 q_3: ┤  dczc ├───────┼────■──────────────────────
      │       │       │    │       ┌───────┐      
 q_4: ┤       ├───────┼────┼───────┤0      ├──────
      │       │┌───┐┌─┴─┐┌─┴─┐┌───┐│  dczc │┌─┐   
a1_0: ┤1      ├┤ X ├┤ X ├┤ X ├┤ X ├┤1      ├┤M├───
      └─┬───┬─┘└─┬─┘└───┘└───┘└─┬─┘└─┬───┬─┘└╥┘┌─┐
a1_1: ──┤ H ├────■──────────────■────┤ H ├───╫─┤M├
        └───┘                        └───┘   ║ └╥┘
c1: 2/═══════════════════════════════════════╩══╩═
                                             0  1

Error syndrome and flag for syndrome 1 :  00


┌───────┐      
 q_0: ─────────────────────────────┤0      ├──────
                                   │       │      
 q_1: ─────────────────────────────┤       ├──────
      ┌───────┐                    │       │      
 q_2: ┤0      ├────────────────────┤       ├──────
      │       │                    │  dczc │      
 q_3: ┤       ├───────■────────────┤       ├──────
      │  dczc │       │            │       │      
 q_4: ┤       ├───────┼────■───────┤       ├──────
      │       │┌───┐┌─┴─┐┌─┴─┐┌───┐│       │┌─┐   
a2_0: ┤1      ├┤ X ├┤ X ├┤ X ├┤ X ├┤1      ├┤M├───
      └─┬───┬─┘└─┬─┘└───┘└───┘└─┬─┘└─┬───┬─┘└╥┘┌─┐
a2_1: ──┤ H ├────■──────────────■────┤ H ├───╫─┤M├
        └───┘                        └───┘   ║ └╥┘
c2: 2/═══════════════════════════════════════╩══╩═
                                             0  1

Error syndrome and flag for syndrome 2 :  00


q_0: ─────────────────────■──────────────────────
                           │       ┌───────┐      
 q_1: ─────────────────────┼───────┤0      ├──────
                           │       │       │      
 q_2: ─────────────────────┼───────┤       ├──────
      ┌───────┐            │       │       │      
 q_3: ┤0      ├────────────┼───────┤  dczc ├──────
      │       │            │       │       │      
 q_4: ┤  dczc ├───────■────┼───────┤       ├──────
      │       │┌───┐┌─┴─┐┌─┴─┐┌───┐│       │┌─┐   
a3_0: ┤1      ├┤ X ├┤ X ├┤ X ├┤ X ├┤1      ├┤M├───
      └─┬───┬─┘└─┬─┘└───┘└───┘└─┬─┘└─┬───┬─┘└╥┘┌─┐
a3_1: ──┤ H ├────■──────────────■────┤ H ├───╫─┤M├
        └───┘                        └───┘   ║ └╥┘
c3: 2/═══════════════════════════════════════╩══╩═
                                             0  1

Error syndrome and flag for syndrome 3 :  01
Syndrome code:  0100
Error detected and fixed.


# Syndrome 1 Extraction Circuit

In [6]:
qc.draw()

NameError: name 'qc' is not defined

# Error Syndrome Extraction Circuit

In [56]:
qr = QuantumRegister(5, 'q')
anc = QuantumRegister(1, 'a')
cr = ClassicalRegister(4, 'c')
        
extract = QuantumCircuit(qr, anc, cr)
extract.append(extract_syndromes, [qr[0], qr[1], qr[2], qr[3], qr[4], anc[0]], [cr[0], cr[1], cr[2], cr[3]])
        
extract.decompose().draw()